In [1]:
import flax.nnx as nn
import torch
from torch.nn import functional as F
import numpy as np
import jax.numpy as jnp
import jax


import tiktoken
enc = tiktoken.get_encoding('gpt2')

q = "Hello, I'm a language model,"
tokens = enc.encode(q)
tokens = jnp.expand_dims(jnp.array(tokens), axis=0)
tokens = jnp.repeat(tokens, 5, axis=0)

from jax_gpt2 import GPT, GPTConfig
model = GPT.from_pretrained('gpt2')
logits = model(tokens)
logits.shape

/var/local/ML/TRAIN/jax/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading weights from pretrained gpt: gpt2
Length of pytorch state dict: 149
Length of prepared JAX modules dict: 76
Transposing  lm_head


(5, 8, 50257)

In [ ]:
import flax.nnx as nn
import torch
from torch.nn import functional as F
import numpy as np
import jax.numpy as jnp
import jax

import tiktoken
enc = tiktoken.get_encoding('gpt2')

q = "Hello, I'm a language model,"
tokens = enc.encode(q)
tokens = jnp.expand_dims(jnp.array(tokens), axis=0)
tokens = jnp.repeat(tokens, 5, axis=0)

from jax_gpt2 import GPT, GPTConfig
model = GPT.from_pretrained('gpt2')
logits = model(tokens)

while tokens.shape[1] < 10: # max_length=30
    # forward the model to get the logits
    logits = model(tokens) # (B, T, vocab_size) 

# Turns out I was ridiculously running a forever loop, tokens.shape[1] never got updated?

In [5]:
import flax.nnx as nn
import torch
from torch.nn import functional as F
import numpy as np
import jax.numpy as jnp
import jax


import tiktoken
enc = tiktoken.get_encoding('gpt2')

q = "Hello, I'm a language model,"
tokens = enc.encode(q)
tokens = jnp.expand_dims(jnp.array(tokens), axis=0)
tokens = jnp.repeat(tokens, 5, axis=0)

from jax_gpt2 import GPT, GPTConfig
model = GPT.from_pretrained('gpt2')

step_key = jax.random.key(0)

while tokens.shape[1] < 30: # max_length=30
    # forward the model to get the logits
    logits = model(tokens) # (B, T, vocab_size) 
    # take the logits at the last position
    logits = logits[:, -1, :] # (B, vocab_size)
    # get the probabilities
    probs = nn.softmax(logits, axis=-1)
    # do top-k sampling of 50 (huggingface pipeline default)
    # topk_probs here becomes (5, 50), topk_indices is (5, 50)
    top_logits, top_tokens = jax.lax.top_k(logits, min(50, logits.shape[-1]))
    step_key, subkey = jax.random.split(step_key)
    token_idx = jax.random.categorical(step_key, top_logits, axis=-1)
    next_token = jnp.take_along_axis(top_tokens, token_idx[:, None], axis=-1).squeeze(-1)
    tokens = jnp.concatenate((tokens, jnp.vstack(next_token)), axis=1)
    # print(f"Updated value of tokens.shape[1]: {tokens.shape[1]}")

# print the generated text

for i in range(5):
    x = tokens[i, :30].tolist()
    decoded = enc.decode(x)
    print(">", decoded)

loading weights from pretrained gpt: gpt2


Length of pytorch state dict: 149
Length of prepared JAX modules dict: 76
Transposing  lm_head
> Hello, I'm a language model, in some of the other parts of the good news at the 'We need the very common use the time and
> Hello, I'm a language model, "No less two. They had in the world. After the one thing it is also the former year.
> Hello, I'm a language model, on the world are the way one of the city (D. They are not be more than and in the
> Hello, I'm a language model, in a good news such far or with a big deal of the time of CUD.
The first to
> Hello, I'm a language model, or the current government will be able to put the same or,

This last few other big big things


# Slowness(Actually infinite loop) fixed, but still jibberish